In [1]:
import pandas as pd
import spacy
import random

In [2]:
#READ NORMALIZED DATA FOR TARGET VERSION OF PIPELINE
df = pd.read_csv("C:\\Users\\brush\\Projekt_NLP\\normalizedResult.tsv", sep="\t")

In [3]:
#READ DATA THAT CONTAINS UNIQUE WORDS FOR EACH TARGET
# onlyPerson=[]
# onlyGroup=[]
# onlyPublic=[]

# personFile = open('onlyPersonWords.txt', 'r', encoding="utf-8")
# temp = personFile.readlines()
# for line in temp:
#     onlyPerson.append(line.replace("\n", ""))
    
# groupFile = open('onlyGroupWords.txt', 'r', encoding="utf-8")
# temp = groupFile.readlines()
# for line in temp:
#     onlyGroup.append(line.replace("\n", ""))

# publicFile = open('onlyPublicWords.txt', 'r', encoding="utf-8")
# temp = publicFile.readlines()
# for line in temp:
#     onlyPublic.append(line.replace("\n", ""))

In [4]:
#CREATE TWO LISTS THAT CONTAIN EACH TEXT OF A TWEET AND ITS TARGET
target = (df.iloc[0:8132 , 2 ].to_list())
text= (df.iloc[0:8132, 1 ].to_list())
print (len(text))
print (len(target))

8132
8132


In [5]:
#ADDING ADDITIONAL WORDS WITH CORRESPONDING TARGETS
# for x in range(0, 2):
#     for word in onlyPerson:
#         text.append(word)
#         target.append(0)
#     for word in onlyGroup:
#         text.append(word)
#         target.append(1)
#     for word in onlyPublic:
#         text.append(word)
#         target.append(2)
# print (len(text))
# print (len(target))

In [6]:
#REMOVAL OF TWEETS WITHOUT ANY CONTENT AND ITS CORRESPONDING ENTRY IN TARGET LIST
counter=0
for t in text:
    if type(t) is float:
        text.remove(t)
        target.remove(target[counter])
    counter+=1

print (len(text))
print(len(target))

8081
8081


In [7]:
# #TOKENISATION OF TWEETS AND LOWERCASING OF WORDS
# array=[]

# nlp = spacy.load("de_core_news_sm")
# for t in text:
#     doc = nlp(t)

#     temp = [token.lemma_.lower() for token in doc if token.is_alpha 
# #             and not token.pos_ == 'ADJ' 
# #             and not token.pos_ == 'CCONJ'
# #             and not token.pos_ == 'SCONJ' 
# #             and not token.pos_ == 'INTJ'
# #             and not token.pos_ == 'PREP'
# #             and not token.pos_ == 'ADV'
# #             and not token.pos_ == 'VERB'
# #               and token.pos_ == "AUX"
# #               or token.pos_ == "DET"
# #               or token.pos_ == "NOUN"
# #               or token.pos_ == "PRON"
# #               or token.pos_ == "PROPN"
# #               or token.pos_ == "VERB"
# #               or token.pos_ == "ADV"
#               ]
#     array.append((" ".join(temp)))

In [8]:
#RANDOMIZER THAT ZIPS TWO CORRESPONDING LISTS TOGETHER AND SHUFFLES BOTH LISTS SYMMETRICALLY
# temporary = list(zip(text, target))
# random.shuffle(temporary)
# text, target = zip(*temporary)
# text, target = list(text), list(target)

In [9]:
#CREATE DATAFRAME THAT CONTAINS TWEET TEXT AND TARGET
df = pd.DataFrame ({'Text': text , 'Target': target })
#print (df)

In [10]:
#SPLIT DATA INTO TRAIN AND TEST DATA
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    text, 
    target, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022, #-> same order of dataset
    stratify=target
)
print("Shape of X_train: ", len(X_train))
print("Shape of X_test: ", len(X_test))

Shape of X_train:  6464
Shape of X_test:  1617


In [11]:
###TARGET SPLITTING OF 0.8 DATA###
targetPerson=[]
targetGroup=[]
targetPublic=[]

counter=0
for t in X_train:
    if y_train[counter] == 0:
        targetPerson.append(t)
    elif y_train[counter] == 1:
        targetGroup.append(t)
    else:
        targetPublic.append(t)
    counter+=1
###TARGET SPLITTING END###

In [12]:
###UNIQUE WORD SPLITTING OF 0.8 DATA###
personWords=[]
groupWords=[]
publicWords=[]

for tweet in targetPerson:
    personWords.append(tweet.split())
personWords = sum(personWords, [])

for tweet in targetGroup:
    groupWords.append(tweet.split())
groupWords = sum(groupWords, [])

for tweet in targetPublic:
    publicWords.append(tweet.split())
publicWords = sum(publicWords, [])

personUnique=[]
groupUnique=[]
publicUnique=[]

for word in personWords:
    if word not in personUnique:
        personUnique.append(word)
        
for word in groupWords:
    if word not in groupUnique:
        groupUnique.append(word)
        
for word in publicWords:
    if word not in publicUnique:
        publicUnique.append(word)
        
onlyPerson=[]

for word in personUnique:
    if word not in groupUnique:
        if word not in publicUnique:
            onlyPerson.append(word)
            
onlyGroup=[]

for word in groupUnique:
    if word not in personUnique:
        if word not in publicUnique:
            onlyGroup.append(word)
            
onlyPublic=[]

for word in publicUnique:
    if word not in personUnique:
        if word not in groupUnique:
            onlyPublic.append(word)
###WORD SPLITTING END###

In [13]:
#y_train.value_counts()

In [14]:
#y_test.value_counts()

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import xgboost as xgb

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     #('KNN', KNeighborsClassifier())
     ('SVM', SVC()),
     #('rfc', RandomForestClassifier()) 
     #('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=42)) 
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#############################################


#print the classfication report BEFORE ANALYZER
classificationBefore = classification_report(y_test, y_pred)

# find accuracy scores
accuracyBefore = accuracy_score(y_test, y_pred)

In [16]:
#4. run analyzer for target correction
# bestAcc=0
# bestX=0
# bestY=0
# y_predCopy = y_pred

# for x in range(1,10):
#     for y in range(1,10):
#         personCount = 0
#         publicCount = 0
#         counter=0
#         personVal=False
#         publicVal=False

#         for p in y_pred:
#             if p == 1:
#                 doc = X_test[counter].split()
#                 for token in doc:
#                     if token in onlyPerson:
#                         personCount+=1
#                     if token in onlyPublic:
#                         publicCount+=1

#                 if personCount >= x:
#                     personVal=True
#                 if publicCount >= y:
#                     publicVal=True

#                 if personVal == True and publicVal == True:
#                     y_predCopy[counter] = 2
#                 if personVal == True:
#                     y_predCopy[counter] = 0
#                 else:
#                     y_predCopy[counter] = 2

#                 personCount = 0
#                 publicCount = 0
#                 personVal=False
#                 publicVal=False
#             counter+=1
        
#         personIst=0
#         groupIst=0
#         publicIst=0
#         for p in y_predCopy:
#             if p == 0:
#                 personIst+=1
#             if p == 1:
#                 groupIst+=1
#             if p == 2:
#                 publicIst+=1
        
#         if personIst != 0 and groupIst != 0 and publicIst !=0:
#             #print the classfication report AFTER ANALYZER
#             classificationAfter = classification_report(y_test, y_predCopy)

#             # find accuracy scores
#             accuracyAfter = accuracy_score(y_test, y_predCopy)
#             if accuracyAfter > bestAcc:
#                 bestAcc = accuracyAfter
#                 bestX = x
#                 bestY = y
# print("Best Acc: " + str(bestAcc))
# print("for:")
# print("X: " + str(bestX) + "     Y: " + str(bestY))

In [17]:
personSoll=0
groupSoll=0
publicSoll=0
for p in y_test:
    if p == 0:
        personSoll+=1
    if p == 1:
        groupSoll+=1
    if p == 2:
        publicSoll+=1
        
personIst=0
groupIst=0
publicIst=0
for p in y_pred:
    if p == 0:
        personIst+=1
    if p == 1:
        groupIst+=1
    if p == 2:
        publicIst+=1
print("Person: " + str(personIst))
print("Group: " + str(groupIst))
print("Public: " + str(publicIst))

Person: 543
Group: 913
Public: 161


In [18]:
#print the classfication report BEFORE ANALYZER
classificationBefore = classification_report(y_test, y_pred)

# find accuracy scores
accuracyBefore = accuracy_score(y_test, y_pred)

In [19]:
#4. run analyzer for target correction
personCount = 0
groupCount = 0
publicCount = 0
counter=0
#best: 3, 3
thresholdPerson=1
thresholdGroup=1
thresholdPublic=1
personVal=False
groupVal=False
publicVal=False
single_adress=["herr", "hr", "frau", "fr", "dame", "mr", "professor", "prof", "dr", "doktor", "doctor", "präsident", "junge"]
single_politicians=["merkel", "steinmeier", "söder", "obama", "putin", "trump", "spahn"]
group_pronouns=["euch", "eure", "euer", "euere", "deren"]
group_politics=["spd", "cdu", "csu", "afd", "grünen", "grüne", "union", "linken", "nazis"]
public_pronouns=["alle", "wir", "menschen"]

for p in y_pred:
    if p == 1:
        doc = X_test[counter].split()
        for token in doc:
            if token in onlyPerson or token in single_adress or token in single_politicians:
                personCount+=1
            if token in onlyGroup or token in group_pronouns or token in group_politics:
                groupCount+=1
            if token in onlyPublic or token in public_pronouns:
                publicCount+=1
                
        if personCount >= thresholdPerson:
            personVal=True
        if groupCount >= thresholdGroup:
            groupVal=True
        if publicCount >= thresholdPublic:
            publicVal=True
            
        if personVal == True and groupVal == True and publicVal == True:
            y_pred[counter] = 2
        if personVal == True and groupVal == True:
            y_pred[counter] = 1
        if personVal == True and publicVal == True:
            y_pred[counter] = 0
        if groupVal == True and publicVal == True:
            y_pred[counter] = 1
        if personVal == True:
            y_pred[counter] = 0
        if groupVal == True:
            y_pred[counter] = 1
        if publicVal == True:
            y_pred[counter] = 2
            
        personCount = 0
        publicCount = 0
        personVal=False
        publicVal=False
    counter+=1

In [20]:
personIst2=0
groupIst2=0
publicIst2=0
for p in y_pred:
    if p == 0:
        personIst2+=1
    if p == 1:
        groupIst2+=1
    if p == 2:
        publicIst2+=1
print("Person: " + str(personIst2))
print("Group: " + str(groupIst2))
print("Public: " + str(publicIst2))

Person: 543
Group: 602
Public: 472


In [21]:
#print the classfication report AFTER ANALYZER
classificationAfter = classification_report(y_test, y_pred)

# find accuracy scores
accuracyAfter = accuracy_score(y_test, y_pred)

In [22]:
print("SOLL-WERT:")
print("Person: " + str(personSoll))
print("Group: " + str(groupSoll))
print("Public: " + str(publicSoll))
print("###############################################")
print("BEFORE ANALZYER:")
print(classificationBefore)
print("The accuracy of prediction is: ", accuracyBefore)
print("IST-WERT DAVOR:")
print("Person: " + str(personIst))
print("Group: " + str(groupIst))
print("Public: " + str(publicIst))
print("###############################################")
print("AFTER ANALYZER:")
print(classificationAfter)
print("The accuracy of prediction is: ", accuracyAfter)
print("IST-WERT DANACH:")
print("Person: " + str(personIst2))
print("Group: " + str(groupIst2))
print("Public: " + str(publicIst2))

SOLL-WERT:
Person: 571
Group: 642
Public: 404
###############################################
BEFORE ANALZYER:
              precision    recall  f1-score   support

           0       0.31      0.30      0.31       571
           1       0.38      0.54      0.45       642
           2       0.22      0.09      0.12       404

    accuracy                           0.34      1617
   macro avg       0.30      0.31      0.29      1617
weighted avg       0.32      0.34      0.32      1617

The accuracy of prediction is:  0.3432282003710575
IST-WERT DAVOR:
Person: 543
Group: 913
Public: 161
###############################################
AFTER ANALYZER:
              precision    recall  f1-score   support

           0       0.31      0.30      0.31       571
           1       0.40      0.37      0.38       642
           2       0.26      0.30      0.28       404

    accuracy                           0.33      1617
   macro avg       0.32      0.32      0.32      1617
weighted avg    